In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px

import plotly.io as pio

import plotly.graph_objects as go
import plotly.figure_factory as ff

import matplotlib.pyplot as plt


In [7]:
test = pd.read_csv("C:/Users/Avery/Documents/GitHub/Regression_proj/dsProject/test_sample.csv")
train = pd.read_csv("C:/Users/Avery/Documents/GitHub/Regression_proj/dsProject/train_sample.csv")

isFraud is the response variable

In [3]:
# train_corr = train_sample.corr()

In [4]:
# # Extract the correlations as a DataFrame
# abscorr = train_corr.abs()

# # Filter the DataFrame to show only correlations greater than 0.7 and not equal to 1
# high_corr = abscorr[(abscorr > 0.7) & (abscorr != 1)]

# # Print the high correlation pairs
# high_corr_dict = (high_corr.stack().drop_duplicates()).to_dict()

# high_corr_dict.keys()

In [5]:
# corrs = {k:v for k, v in high_corr_dict.items() if 'C1' in k}
# corrs


In [10]:
test.drop(columns='Unnamed: 0',inplace= True)